In [ ]:
from gerrychain import Graph, Partition, GeographicPartition, Election, MarkovChain, constraints, accept
from gerrychain.updaters import Tally
from gerrychain.proposals import recom
from functools import partial
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import random
import numpy as np
import geopandas as gpd
import pandas
from gerrychain.metrics import polsby_popper
import networkx as nx
import math
import json
import maup
from itertools import groupby

In [ ]:
# adapted from existing from_districtr_file function in GerryChain
def from_districtr_file(cls, graph, districtr_file, updaters=None):
        """Create a Partition from a districting plan created with `Districtr`_,
        a free and open-source web app created by MGGG for drawing districts.

        The provided ``graph`` should be created from the same shapefile as the
        Districtr module used to draw the districting plan. These shapefiles may
        be found in a repository in the `mggg-states`_ GitHub organization, or by
        request from MGGG.

        .. _`Districtr`: https://mggg.org/Districtr
        .. _`mggg-states`: https://github.com/mggg-states

        :param graph: :class:`~gerrychain.Graph`
        :param districtr_file: the path to the ``.json`` file exported from Districtr
        :param updaters: dictionary of updaters
        """
        with open(districtr_file) as f:
            districtr_plan = json.load(f)

        id_column_key = districtr_plan["idColumn"]["key"]
        districtr_assignment = districtr_plan["assignment"]
        try:
            node_to_id = {node: str(graph.nodes[node][id_column_key]) for node in graph}
        except KeyError:
            raise TypeError(
                "The provided graph is missing the {} column, which is "
                "needed to match the Districtr assignment to the nodes of the graph."
            )

        assignment = {}
        dist_populations = {}
        bad_nodes = {} #maps nodes without assignments to adjacent nodes with multiplicity
        for node in graph:
            if node_to_id[node] not in districtr_assignment.keys():
                bad_nodes[node] = []
                for node2 in graph:
                    if (node, node2) in graph.edges or (node2, node) in graph.edges:
                        bad_nodes[node].append(node2)
            else:
                assignment[node] = districtr_assignment[node_to_id[node]][0]
                if districtr_assignment[node_to_id[node]][0] not in dist_populations.keys():
                    dist_populations[districtr_assignment[node_to_id[node]][0]] = 0
                dist_populations[districtr_assignment[node_to_id[node]][0]] += graph.nodes[node]['TOTPOP']
        
        # assigns districts to nodes without assignments
        while len(bad_nodes) > 0:
            to_pop = []
            for node in bad_nodes.keys():
                possible_districts = []
                for node2 in bad_nodes[node]:
                    if node2 in assignment.keys():
                        possible_districts.append(assignment[node2])
                if len(possible_districts) > 0: 
                    # assign a district to this node
                    # find the district that has the lower population or if tied, the more compact one
                    district = possible_districts[0]
                    for dist in possible_districts:
                        if dist_populations[dist] < dist_populations[district]:
                            district = dist
                    assignment[node] = district
                    # update populations
                    dist_populations[district] += graph.nodes[node]['TOTPOP']
                    # remove this node from bad_nodes
                    to_pop.append(node)
            for node in to_pop:
                bad_nodes.pop(node)                    
                
        return cls(graph, assignment, updaters)

In [ ]:
graph = Graph.from_file("../states/MI/MI.shp")

In [ ]:
# good one; no holes
# plans downloaded from https://www.michigan-mapping.org/#gallery
partition1 = from_districtr_file(GeographicPartition, graph, districtr_file="districtr-plan-d9abbad0.json", updaters=None)

In [ ]:
partition1.plot(cmap='tab20')

In [ ]:
# bad one; holes
partition2 = from_districtr_file(GeographicPartition, graph, districtr_file="districtr-plan-a36af749.json", updaters=None)
partition2.plot(cmap='tab20')